In [ ]:
import pickle
from flask import Flask, render_template, request
import numpy as np
import tensorflow as tf
import pandas as pd
import cv2
from sklearn.preprocessing import StandardScaler

In [ ]:
app=Flask(__name__)

In [ ]:
from tensorflow.keras.models import load_model
humantracker = load_model('humantracker.h5')

In [ ]:
@app.route('/')
def home():
    return render_template('index.html')

In [ ]:
@app.route('/predict', methods=['POST'])
def predictt():
    video= request.files['video_input']
    print(video.filename)
    video_path = "./videos/" + video.filename
    video.save(video_path)
    cap = cv2.VideoCapture(video_path)
    if (cap.isOpened() == False):
        print("Error opening video stream or file")
    while cap.isOpened():
        _ , frame = cap.read()
        frame = frame[50:500, 50:500,:]
        
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resized = tf.image.resize(rgb, (120,120))
        
        yhat = humantracker.predict(np.expand_dims(resized/255,0))
        sample_coords = yhat[1][0]
        print(yhat[0])
    
        if yhat[0] >0.5: 
        # Controls the main rectangle
            cv2.rectangle(frame, 
                          tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                          tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                                (255,0,0), 2)
        # Controls the label rectangle
            cv2.rectangle(frame, 
                          tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                        [0,-30])),
                          tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                        [80,0])), 
                                (255,0,0), -1)
        
        # Controls the text rendered
            cv2.putText(frame, 'Human', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                   [0,-5])),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
        cv2.imshow('humantracker', frame)
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    return render_template('index.html')

In [ ]:
if __name__ == '__main__':
    app.run(host='localhost', port=80)